# IALS implicit

## Import

In [1]:
## Allow more than one output for a single code cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [2]:
import pandas as pd
import scipy.sparse as sps
import numpy as np
import os

## Set the numpy random seed
SEED = 42
np.random.seed(SEED)

os.getcwd()

'/home/alessio/Scrivania/RecSysChallenge2021'

In [3]:
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

from Evaluation.Evaluator import EvaluatorHoldout

from Recommenders.Recommender_import_list import *

from Recommenders.DataIO import DataIO

Tensorflow is not available


In [4]:
## Utility Functions
from Dataset.load_data import load_data

## Data Loading and Split

In [5]:
URM_all, ICM_dict = load_data()

In [6]:
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)

URM_aug_train = sps.vstack([URM_train.copy().tocoo(), 
                            #ICM_dict['ICM_genre'].T.tocoo(),
                            ICM_dict['ICM_subgenre'].T.tocoo(), 
                            #ICM_dict['ICM_event'].T.tocoo(), 
                            ICM_dict['ICM_channel'].T.tocoo()], format='csr')

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10], exclude_seen = True)

EvaluatorHoldout: Ignoring 13646 ( 0.0%) Users that have less than 1 test interactions


## Optimization

In [6]:
!ls "./result_experiments/IALS"

IALS_AUG_genre_subgenre_channel_range_improved	 IALS_NO_AUG
IALS_AUG_genre_subgenre_channel_range_improved2  IALS_NO_AUG_2
IALS_AUG_rangeImproved				 IALS_NO_AUG_3
IALS_AUG_subgenre_channel_range_improved


In [13]:
output_folder_path = "result_experiments/IALS/IALS_NO_AUG/"

data_loader = DataIO(folder_path = output_folder_path)
search_metadata = data_loader.load_data(IALSRecommender.RECOMMENDER_NAME + "_metadata.zip")

#search_metadata.keys()

hyperparameters_df = search_metadata["hyperparameters_df"]
result_on_validation_df = search_metadata["result_on_validation_df"]

hyperparameters_df['MAP'] = result_on_validation_df['MAP'].values.tolist()

threshold = 0.23

hyperparameters_df[hyperparameters_df['MAP'] > threshold]
print('Max value for the range: ', max(hyperparameters_df[hyperparameters_df['MAP'] > threshold].num_factors))
print('Min value for the range: ', min(hyperparameters_df[hyperparameters_df['MAP'] > threshold].num_factors))
print('Best MAP: ', max(hyperparameters_df['MAP']))
hyperparameters_df[max(hyperparameters_df['MAP']) == hyperparameters_df['MAP']]

,num_factors,epochs,confidence_scaling,alpha,epsilon,reg,MAP
17,52,15,linear,1.334607,0.17159,0.000584,0.233916
25,61,45,log,4.43449,10.0,0.000609,0.230052
36,36,35,linear,0.394312,0.275704,0.000606,0.232647
37,57,30,log,0.802954,0.514866,0.000552,0.233861


Max value for the range:  61
Min value for the range:  36
Best MAP:  0.23391613682944476


,num_factors,epochs,confidence_scaling,alpha,epsilon,reg,MAP
17,52,15,linear,1.334607,0.17159,0.000584,0.233916


In [7]:
output_folder_path = "result_experiments/IALS/IALS_NO_AUG_2/" # range num_factors 30->80

data_loader = DataIO(folder_path = output_folder_path)
search_metadata = data_loader.load_data(IALSRecommender.RECOMMENDER_NAME + "_metadata.zip")

#search_metadata.keys()

hyperparameters_df = search_metadata["hyperparameters_df"]
result_on_validation_df = search_metadata["result_on_validation_df"]

hyperparameters_df['MAP'] = result_on_validation_df['MAP'].values.tolist()

threshold = 0.

hyperparameters_df[hyperparameters_df['MAP'] > threshold]
print('Max value for the range: ', max(hyperparameters_df[hyperparameters_df['MAP'] > threshold].num_factors))
print('Min value for the range: ', min(hyperparameters_df[hyperparameters_df['MAP'] > threshold].num_factors))
print('Best MAP: ', max(hyperparameters_df['MAP']))
hyperparameters_df[max(hyperparameters_df['MAP']) == hyperparameters_df['MAP']]

,num_factors,epochs,confidence_scaling,alpha,epsilon,reg,MAP
0,49,5,log,15.746724,0.020754,0.001112,0.057425
1,67,65,linear,0.002156,0.003963,0.000172,0.214381
2,45,5,log,10.10879,0.010094,0.000035,0.076376
3,62,185,log,0.001161,5.656201,0.0005,0.217440
4,64,50,log,0.064651,4.544871,0.007812,0.215799
5,39,30,log,0.003211,0.022768,0.007263,0.230985
6,41,170,log,11.146458,1.264986,0.000064,0.206778


Max value for the range:  67
Min value for the range:  39
Best MAP:  0.2309848313012254


,num_factors,epochs,confidence_scaling,alpha,epsilon,reg,MAP
5,39,30,log,0.003211,0.022768,0.007263,0.230985


In [9]:
output_folder_path = "result_experiments/IALS/IALS_NO_AUG_3/" # range num_factors 30->80

data_loader = DataIO(folder_path = output_folder_path)
search_metadata = data_loader.load_data(IALSRecommender.RECOMMENDER_NAME + "_metadata.zip")

#search_metadata.keys()

hyperparameters_df = search_metadata["hyperparameters_df"]
result_on_validation_df = search_metadata["result_on_validation_df"]

hyperparameters_df['MAP'] = result_on_validation_df['MAP'].values.tolist()

threshold = 0.23

hyperparameters_df[hyperparameters_df['MAP'] > threshold]
print('Max value for the range: ', max(hyperparameters_df[hyperparameters_df['MAP'] > threshold].num_factors))
print('Min value for the range: ', min(hyperparameters_df[hyperparameters_df['MAP'] > threshold].num_factors))
print('Best MAP: ', max(hyperparameters_df['MAP']))
hyperparameters_df[max(hyperparameters_df['MAP']) == hyperparameters_df['MAP']]

,num_factors,epochs,confidence_scaling,alpha,epsilon,reg,MAP
17,52,15,linear,1.334607,0.17159,0.000584,0.233916
25,61,45,log,4.43449,10.0,0.000609,0.230052
36,36,35,linear,0.394312,0.275704,0.000606,0.232647
37,57,30,log,0.802954,0.514866,0.000552,0.233861


Max value for the range:  61
Min value for the range:  36
Best MAP:  0.23391613682944476


,num_factors,epochs,confidence_scaling,alpha,epsilon,reg,MAP
17,52,15,linear,1.334607,0.17159,0.000584,0.233916


In [7]:
output_folder_path = "result_experiments/IALS/IALS_AUG_rangeImproved/"

data_loader = DataIO(folder_path = output_folder_path)
search_metadata = data_loader.load_data(IALSRecommender.RECOMMENDER_NAME + "_metadata.zip")

hyperparameters_df = search_metadata["hyperparameters_df"]
result_on_validation_df = search_metadata["result_on_validation_df"]

hyperparameters_df['MAP'] = result_on_validation_df['MAP'].values.tolist()

threshold = 0.23

hyperparameters_df[hyperparameters_df['MAP'] > threshold]
print('Max value for the range: ', max(hyperparameters_df[hyperparameters_df['MAP'] > threshold].num_factors))
print('Min value for the range: ', min(hyperparameters_df[hyperparameters_df['MAP'] > threshold].num_factors))
print('Best MAP: ', max(hyperparameters_df['MAP']))
hyperparameters_df[max(hyperparameters_df['MAP']) == hyperparameters_df['MAP']]

,num_factors,epochs,confidence_scaling,alpha,epsilon,reg,MAP
12,20,25,linear,0.001,5.532497,0.002147,0.230450
14,32,35,linear,0.010477,0.373426,0.005897,0.231824
15,37,45,log,0.001946,0.001079,0.000179,0.232057
17,20,10,log,0.009769,0.001,0.00001,0.230765
18,44,45,linear,0.0719,0.001554,0.005092,0.230649
19,48,35,log,0.010645,0.001,0.00001,0.230431


Max value for the range:  48
Min value for the range:  20
Best MAP:  0.23205652450046393


,num_factors,epochs,confidence_scaling,alpha,epsilon,reg,MAP
15,37,45,log,0.001946,0.001079,0.000179,0.232057


In [10]:
output_folder_path = "result_experiments/IALS/IALS_AUG_subgenre_channel_range_improved/"

data_loader = DataIO(folder_path = output_folder_path)
search_metadata = data_loader.load_data(IALSRecommender.RECOMMENDER_NAME + "_metadata.zip")

hyperparameters_df = search_metadata["hyperparameters_df"]
result_on_validation_df = search_metadata["result_on_validation_df"]

hyperparameters_df['MAP'] = result_on_validation_df['MAP'].values.tolist()

threshold = 0.23

hyperparameters_df[hyperparameters_df['MAP'] > threshold]
print('Max value for the range: ', max(hyperparameters_df[hyperparameters_df['MAP'] > threshold].num_factors))
print('Min value for the range: ', min(hyperparameters_df[hyperparameters_df['MAP'] > threshold].num_factors))
print('Best MAP: ', max(hyperparameters_df['MAP']))
hyperparameters_df[max(hyperparameters_df['MAP']) == hyperparameters_df['MAP']]

,num_factors,epochs,confidence_scaling,alpha,epsilon,reg,MAP
3,61,80,linear,0.354668,0.298795,0.000104,0.230335
7,53,25,linear,0.549308,0.041181,0.000029,0.234725
12,30,95,linear,0.003879,0.07972,0.001368,0.231864
13,33,20,linear,0.271473,0.00562,0.000944,0.233298
14,20,10,linear,0.008081,0.600159,0.007789,0.230454
16,20,20,log,0.426688,10.0,0.00001,0.230517
18,20,30,linear,0.353464,10.0,0.00001,0.230424
19,20,20,log,0.001,10.0,0.000014,0.230450
20,20,10,linear,0.001,10.0,0.01,0.230489
21,20,10,log,0.054691,10.0,0.00001,0.231031


Max value for the range:  61
Min value for the range:  20
Best MAP:  0.23472468820446227


,num_factors,epochs,confidence_scaling,alpha,epsilon,reg,MAP
7,53,25,linear,0.549308,0.041181,0.000029,0.234725


In [11]:
output_folder_path = "result_experiments/IALS/IALS_AUG_genre_subgenre_channel_range_improved/"

data_loader = DataIO(folder_path = output_folder_path)
search_metadata = data_loader.load_data(IALSRecommender.RECOMMENDER_NAME + "_metadata.zip")

hyperparameters_df = search_metadata["hyperparameters_df"]
result_on_validation_df = search_metadata["result_on_validation_df"]

hyperparameters_df['MAP'] = result_on_validation_df['MAP'].values.tolist()

threshold = 0.23

hyperparameters_df[hyperparameters_df['MAP'] > threshold]
print('Max value for the range: ', max(hyperparameters_df[hyperparameters_df['MAP'] > threshold].num_factors))
print('Min value for the range: ', min(hyperparameters_df[hyperparameters_df['MAP'] > threshold].num_factors))
print('Best MAP: ', max(hyperparameters_df['MAP']))
hyperparameters_df[max(hyperparameters_df['MAP']) == hyperparameters_df['MAP']]

,num_factors,epochs,confidence_scaling,alpha,epsilon,reg,MAP
0,44,45,linear,0.963314,0.191079,0.000021,0.233791
2,31,75,log,0.020209,0.003654,0.001082,0.232963
4,50,70,log,0.010034,0.014398,0.000054,0.230521
7,65,80,linear,0.549096,0.193704,0.003595,0.232578
11,36,25,linear,0.320259,0.009008,0.000676,0.233280
12,36,70,linear,0.010658,0.031745,0.00421,0.232733
13,32,75,linear,1.334469,0.151492,0.000013,0.231241
15,31,50,linear,0.48299,10.0,0.00001,0.233082
16,20,40,log,0.001,10.0,0.00001,0.231037
17,20,25,log,0.002718,0.001,0.000063,0.230948


Max value for the range:  65
Min value for the range:  20
Best MAP:  0.23379091843880256


,num_factors,epochs,confidence_scaling,alpha,epsilon,reg,MAP
0,44,45,linear,0.963314,0.191079,0.000021,0.233791


In [12]:
output_folder_path = "result_experiments/IALS/IALS_AUG_genre_subgenre_channel_range_improved2/"

data_loader = DataIO(folder_path = output_folder_path)
search_metadata = data_loader.load_data(IALSRecommender.RECOMMENDER_NAME + "_metadata.zip")

hyperparameters_df = search_metadata["hyperparameters_df"]
result_on_validation_df = search_metadata["result_on_validation_df"]

hyperparameters_df['MAP'] = result_on_validation_df['MAP'].values.tolist()

threshold = 0.23

hyperparameters_df[hyperparameters_df['MAP'] > threshold]
print('Max value for the range: ', max(hyperparameters_df[hyperparameters_df['MAP'] > threshold].num_factors))
print('Min value for the range: ', min(hyperparameters_df[hyperparameters_df['MAP'] > threshold].num_factors))
print('Best MAP: ', max(hyperparameters_df['MAP']))
hyperparameters_df[max(hyperparameters_df['MAP']) == hyperparameters_df['MAP']]

,num_factors,epochs,confidence_scaling,alpha,epsilon,reg,MAP
7,36,65,log,0.043179,0.003408,0.001427,0.233326
9,38,10,log,0.375048,0.151208,0.000014,0.233537
10,67,20,log,0.192931,0.042887,0.005528,0.231678
11,56,35,log,0.477033,0.115511,0.001459,0.234782
12,27,50,linear,0.00242,3.47859,0.00852,0.232996
13,50,40,log,0.858279,1.663407,0.002681,0.234459
15,20,35,log,0.001,10.0,0.01,0.231092
16,20,50,linear,0.003939,0.001,0.01,0.230973
18,20,15,log,0.022003,10.0,0.01,0.231106
19,20,50,log,0.014449,0.055212,0.01,0.231064


Max value for the range:  67
Min value for the range:  20
Best MAP:  0.23478184781167408


,num_factors,epochs,confidence_scaling,alpha,epsilon,reg,MAP
11,56,35,log,0.477033,0.115511,0.001459,0.234782


In [14]:
output_folder_path = "result_experiments/IALS_NO_AUG_2/" 

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

n_cases = 50  # 50 with 30% random is a good number
n_random_starts = int(n_cases*0.3)
metric_to_optimize = "MAP"   
cutoff_to_optimize = 10

In [ ]:
from functools import partial
import os, multiprocessing

from HyperparameterTuning.run_hyperparameter_search import runHyperparameterSearch_Collaborative
from Recommenders.MatrixFactorization.IALSRecommender_implicit import IALSRecommender_implicit
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender

runHyperparameterSearch_Collaborative(IALSRecommender,
                                      URM_train = URM_train,
                                      URM_train_last_test = None,
                                      metric_to_optimize = metric_to_optimize,
                                      cutoff_to_optimize = cutoff_to_optimize,
                                      n_cases = n_cases,
                                      n_random_starts = n_random_starts,
                                      evaluator_validation_earlystopping = evaluator_validation,
                                      evaluator_validation = evaluator_validation,
                                      evaluator_test = None,
                                      output_folder_path = output_folder_path,
                                      resume_from_saved = True,
                                      similarity_type_list = None,
                                      parallelizeKNN = True)